In [1]:
import pandas as pd
df=pd.read_csv('/Users/ashishtrehan/Downloads/parsed_chicken.csv')

/Library/Python/2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [4]:
df.shape

(102066, 9)

In [5]:
df.head()

,ID,len,num_pad,num_groups,seqs,seq_len,seqs_even,identif,snips
0,0,1704,79,568,atggagaaaatagtgcttcttcttgcaatagtcagccttgttaaaa...,atg,atggagaaaatagtgcttcttcttgcaatagtcagccttgttaaaa...,Seqnum1,atg
1,0,1704,79,568,atggagaaaatagtgcttcttcttgcaatagtcagccttgttaaaa...,atg,atggagaaaatagtgcttcttcttgcaatagtcagccttgttaaaa...,Seqnum1,gag
2,0,1704,79,568,atggagaaaatagtgcttcttcttgcaatagtcagccttgttaaaa...,atg,atggagaaaatagtgcttcttcttgcaatagtcagccttgttaaaa...,Seqnum1,aaa
3,0,1704,79,568,atggagaaaatagtgcttcttcttgcaatagtcagccttgttaaaa...,atg,atggagaaaatagtgcttcttcttgcaatagtcagccttgttaaaa...,Seqnum1,ata
4,0,1704,79,568,atggagaaaatagtgcttcttcttgcaatagtcagccttgttaaaa...,atg,atggagaaaatagtgcttcttcttgcaatagtcagccttgttaaaa...,Seqnum1,gtg


In [2]:
import pandas as pd
import numpy as np

In [17]:
df1=pd.read_csv('/Users/ashishtrehan/Downloads/FLU_Challenge_vietnam.csv')

In [12]:
#df1.groupby(['seqs']).size().to_csv()

In [15]:
df.columns

Index([u'ID', u'len', u'num_pad', u'num_groups', u'seqs', u'seq_len',
       u'seqs_even', u'identif', u'snips'],
      dtype='object')

In [19]:
df1.head()

,ID,seq,s,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 584,Unnamed: 585,Unnamed: 586,Unnamed: 587,Unnamed: 588,Unnamed: 589,Unnamed: 590,Unnamed: 591,Unnamed: 592,Unnamed: 593
0,0,agc,aaa,agc,agg,ggt,cca,atc,tgt,caa,...,ttg,tag,tta,aaa,aca,ccc,ttg,ttt,cta,ct
1,0,atg,gag,aaa,ata,gtg,ctt,ctt,ctt,gca,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,atg,gag,aaa,ata,gtg,ctt,ctt,ctt,gca,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,atg,gag,aaa,ata,gtg,ctt,ctt,ctt,gca,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,atg,gag,aaa,ata,gtg,ctt,ctt,ctt,gca,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
trainFeatures = df1.drop(['ID','seq','s'], axis=1)

In [24]:
trainFeatures.groupby(['Unnamed: 3']).size()

Unnamed: 3
aaa    291
aac     16
aag     12
aat      9
aca      1
aga     12
agc     92
atc     48
atg      3
att      4
caa      4
cag      2
cct      1
cta      1
ctc      3
ctg     47
gaa      2
gag     21
gat      1
gca      2
gct      1
gga      3
gta      2
gtc      3
gtt      1
tat      3
tca     11
tct     10
tgc      1
tgg      1
tgt     20
tta      1
ttg      1
dtype: int64

In [21]:
from sklearn.decomposition import PCA, FactorAnalysis
pca = PCA(whiten=True)
pca_1=pca.fit(trainFeatures)

ValueError: could not convert string to float: ct

In [23]:
class TreeFS(sklearn.base.BaseEstimator):
    
    def __init__(self, n_features=100):
        self.forest = ExtraTreesClassifier(n_estimators=100)
        self.n_features = n_features
    
    def fit(self, df, y=None):
        self.forest.fit(df, y)
        self.importances = self.forest.feature_importances_
        self.indices = np.argsort(-self.importances)[0:self.n_features]
        self.indices.sort()
        return self
        
    def transform(self, X, y=None):   
        return X[:, self.indices]
        
class Ngrams(sklearn.base.BaseEstimator):
    def __init__(self, ngrams=(1,1),min_df=1, name="ngram"):
        self.min_df = min_df
        self.ngrams = ngrams
        self.name = name
        
    def get_feature_names(self):
        return self.feature_names_
    
    def _set_feature_names(self, features):
        return [(self.name, x) for x in features]
        
        
    def fit(self, df,y=None):
        self.tfidf = CountVectorizer(ngram_range=self.ngrams, min_df=self.min_df, max_df=0.9,token_pattern="[a-zA-Z]{2,15}")
                                    #vocabulary=['New Girl'])
                                     #stop_words="english")
                                     #token_pattern="[a-zA-Z]{2,15}")
        self.tfidf.fit(df)
        self.feature_names_ = self._set_feature_names(self.tfidf.get_feature_names())
        return self
    
    def transform(self, df,y=None):
        return binarize(self.tfidf.transform(df).todense())

class OneGrams(Ngrams):
    
    def __init__(self, min_df=1):
        super(OneGrams, self).__init__(min_df=min_df, name="onegram")
        

class MGrams(Ngrams):
    
    def __init__(self, ngrams=(2,2), min_df=1):
        super(MGrams, self).__init__(ngrams=ngrams,min_df=min_df,name="mgram")
        
    def _set_feature_names(self, features):
        return [(self.name, "[, .:;/-]+".join(x.split(" "))) for x in features]
    

class FeaturePipeline(FeatureUnion):
    
    def __init__(self, transformer_list, n_jobs=1, transformer_weights=None):
        super(FeaturePipeline, self).__init__(transformer_list, n_jobs, transformer_weights)
        
    def get_features(self):
        feature_names = []
        for name, trans in self.transformer_list:
            feature_names.extend(trans.get_feature_names())
        return feature_names

forest = TreeFS(n_features=800)    
svm = LinearSVC(fit_intercept=False, random_state=1)
fp = FeaturePipeline(sklearn.pipeline._name_estimators([OneGrams(min_df=2), MGrams((2,2), min_df=2), MGrams((3,3),min_df=2)]))
p = Pipeline([("features",fp),("forest",forest),("svm",svm)])
print p

NameError: name 'sklearn' is not defined

In [27]:
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
X, y = make_friedman1(n_samples=50, n_features=10, random_state=0)
estimator = SVR(kernel="linear")
selector = RFE(estimator, 5, step=1)
selector = selector.fit(X, y)
selector.support_ 
selector.ranking_

array([1, 1, 1, 1, 1, 6, 4, 3, 2, 5])

In [28]:
X

array([[ 0.5488135 ,  0.71518937,  0.60276338,  0.54488318,  0.4236548 ,
         0.64589411,  0.43758721,  0.891773  ,  0.96366276,  0.38344152],
       [ 0.79172504,  0.52889492,  0.56804456,  0.92559664,  0.07103606,
         0.0871293 ,  0.0202184 ,  0.83261985,  0.77815675,  0.87001215],
       [ 0.97861834,  0.79915856,  0.46147936,  0.78052918,  0.11827443,
         0.63992102,  0.14335329,  0.94466892,  0.52184832,  0.41466194],
       [ 0.26455561,  0.77423369,  0.45615033,  0.56843395,  0.0187898 ,
         0.6176355 ,  0.61209572,  0.616934  ,  0.94374808,  0.6818203 ],
       [ 0.3595079 ,  0.43703195,  0.6976312 ,  0.06022547,  0.66676672,
         0.67063787,  0.21038256,  0.1289263 ,  0.31542835,  0.36371077],
       [ 0.57019677,  0.43860151,  0.98837384,  0.10204481,  0.20887676,
         0.16130952,  0.65310833,  0.2532916 ,  0.46631077,  0.24442559],
       [ 0.15896958,  0.11037514,  0.65632959,  0.13818295,  0.19658236,
         0.36872517,  0.82099323,  0.09710128